# Load Predictions to the FHIR server

## Get DB connection

In [1]:
def getConnection():

    import psycopg2

    # Connect to postgres with a copy of the MIMIC-IV database
    con = psycopg2.connect(
        dbname='mimic4',
        user='postgres',
        host='superbugai.erc.monash.edu',
        port=5434,
        password='mysecretpassword'
        )

    return con

## Read the data matrix

In [2]:
import pandas as pd


con = getConnection()
dataDf = pd.read_sql("select * from omop_test_20220817.data_matrix", con=con)

/tmp/ipykernel_444831/1215489904.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataDf = pd.read_sql("select * from omop_test_20220817.data_matrix", con=con)


## Push mortality prediction results to FHIR

In [9]:
def convertIdFromOmopToFhir(omopId):
    omopId = 'm' + str(omopId)[1:] if str(omopId).startswith('-') else 'p' + str(omopId)[1:]
    return omopId

In [23]:
template = {
    "resourceType" : "RiskAssessment",
    "status" : "final",
    "subject" : {
      "reference" : ""
    },
    "occurrenceDateTime" : "",
    "prediction" : {
        "outcome" : {
            "text" : "7 day Mortality Prediction"
        },
        "probabilityDecimal" : ""
    },
    "note" : {
        "text" : ""
    }
}


In [41]:
import json
import requests

from tqdm import tqdm
from datetime import datetime


for person_id in tqdm(dataDf.person_id):
    for i in [7, 14, 21, 30, 60, 90, 120]:
        response = requests.post(
            url='http://localhost:5001/predict_mortality', json={"id":person_id, "targetStart":0, "targetEnd":i, "windowEnd":24}
            )
        score = json.loads(response.text)['score']
        if score:
            template['prediction']['probabilityDecimal'] = round(float(score), 4)
            template['subject']['reference'] = 'Patient/' + convertIdFromOmopToFhir(person_id)
            template['occurrenceDateTime'] = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
            template['note']['text'] = "Model: XGBoost Ensemble, Vitals Before: 0, Vitals After: 72, Labs Before: 0, Labs After: 72, Data Window: (0, 24), Target Start: 0, Target End: " + str(i)
            requests.post(url='http://superbugai.erc.monash.edu:8082/fhir/RiskAssessment/', json=template, headers={"Content-Type": "application/fhir+json"})


100%|██████████| 2483/2483 [9:01:02<00:00, 13.07s/it]  
